In [1]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')

In [2]:
%load_ext autotime

In [3]:
import magellan as mg

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


time: 2.48 s


In [25]:
from magellan import MTable
import pandas as pd
from magellan.blockercombiner.blockercombiner import *
def cbn_via_union(blocker_output_list):
    """
    Combine blocker outputs by unioning ltable, rtable ids in candidate set

    Parameters
    ----------
    blocker_output_list : list
        List of blocker outputs

    Returns
    -------
    combined_blocker_output : MTable
        With combined blocker outputs

    Notes
    -----
    Combined_blocker_output contains the following attributes
    * _id
    * combined id pairs (ltable.id, rtabled.id) from list of blocker outputs
    * union of non-id attributes from each of blocker output
    """
    ltable, rtable = lr_tables(blocker_output_list)
    # get the attribute names in blocker output that represents ltable, rtable
    l_key = 'ltable.' + ltable.get_key()
    r_key = 'rtable.' + rtable.get_key()

    # l0 = blocker_output_list[0]
    # l_key = l0.get_property('foreign_key_ltable')
    # r_key = l0.get_property('foreign_key_rtable')
    # lfid_idx = ltable.get_attr_names().index(ltable.get_key())
    l_df = ltable.to_dataframe()
    r_df = rtable.to_dataframe()

    col_names = ['ltable.'+c for c in l_df.columns]
    l_df.columns = col_names
    col_names = ['rtable.'+c for c in r_df.columns]
    r_df.columns = col_names

    l_dict = {}
    # get the union of attribute names from blocker output list
    col_set = set([x for c in blocker_output_list for x in c.columns])
    l_col, r_col = lr_cols(col_set)

    m_lcol = ['ltable.' + c for c in l_col]
    m_rcol = ['rtable.' + c for c in r_col]
    for i, r in l_df.iterrows():
        l_dict[r[l_key]] = r[m_lcol]
    r_dict = {}
    for i, r in r_df.iterrows():
        r_dict[r[r_key]] = r[m_rcol]

    # get id pairs
    id_set = []

    for c in blocker_output_list:
        lfid_idx = c.get_attr_names().index(c.get_property('foreign_key_ltable'))
        rfid_idx = c.get_attr_names().index(c.get_property('foreign_key_rtable'))

        for r in c.itertuples(index=False):
            id_set.append((r[lfid_idx], r[rfid_idx]))
    id_set = set(id_set)



    # # get the set of id pairs from all blocker output list
    # id_set = set([(r[l_key], r[r_key]) for c in blocker_output_list for i, r in c.iterrows()])


    # # convert ltable, rtable to dfs and set index
    # l_df = ltable.to_dataframe()
    # l_df.set_index(ltable.get_key(), inplace=True, drop=False)
    # r_df = rtable.to_dataframe()
    # r_df.set_index(rtable.get_key(), inplace=True, drop=False)

    # get the l_col, r_col from ltable and rtable respectively
    dict_list = []
    if mg._progbar:
            bar = pyprind.ProgBar(len(id_set))
    l_dict_list = []
    r_dict_list = []
    for x in id_set:
        #d = get_dict (l_dict[x[0]], r_dict[x[1]], l_col, r_col)
        l_dict_list.append(l_dict[x[0]])
        r_dict_list.append(r_dict[x[1]])

        if mg._progbar:
            bar.update()
    # dict_list = [get_dict(l_dict[x[0]], r_dict[x[1]], l_col, r_col) for x in id_set]

    # get the final column names for output table
    f_cols = fin_cols(l_col, r_col, ltable.get_key(), rtable.get_key())

    left_consol_table = pd.DataFrame(l_dict_list)
    right_consol_table = pd.DataFrame(r_dict_list)
#     return left_consol_table, right_consol_table
    if len(left_consol_table) > 0 and len(right_consol_table) > 0:
        left_consol_table.reset_index(inplace=True, drop=True)
        right_consol_table.reset_index(inplace=True, drop=True)

        table = pd.concat([left_consol_table, right_consol_table], axis=1)
        table.reset_index(inplace=True, drop=True)
        table.sort([l_key, r_key], inplace=True)
        table.reset_index(inplace=True, drop=True)
        table = MTable(table[f_cols])
    else:
        table = MTable([], columns=f_cols)

    # project df and convert to MTable
    table.set_property('ltable', ltable)
    table.set_property('rtable', rtable)
    table.set_property('foreign_key_ltable', 'ltable.'+ltable.get_key())
    table.set_property('foreign_key_rtable', 'rtable.'+rtable.get_key())


    return id_set

time: 125 ms


In [14]:
from magellan import MTable
import pandas as pd
from magellan.blockercombiner.blockercombiner import *

def cbn_via_union_v2(blocker_output_list):
    ltable, rtable = lr_tables(blocker_output_list)
    # get the attribute names in blocker output that represents ltable, rtable
    l_key = 'ltable.' + ltable.get_key()
    r_key = 'rtable.' + rtable.get_key()

    l_df = ltable.to_dataframe()
    r_df = rtable.to_dataframe()

    # get the union of attribute names from blocker output list
    col_set = set([x for c in blocker_output_list for x in c.columns])
    l_col, r_col = lr_cols(col_set)
    l_col = list(l_col)
    r_col = list(r_col)
    
    l_df = l_df[l_col] # minimally the projection must contain id column
    r_df = r_df[r_col]
        
    col_names = ['ltable.'+c for c in l_df.columns]
    l_df.columns = col_names
    col_names = ['rtable.'+c for c in r_df.columns]
    r_df.columns = col_names


    
    l_df.set_index(l_key, inplace=True, drop=False)
    r_df.set_index(r_key, inplace=True, drop=False)
    
    
    # get id pairs
    id_set = []

    for c in blocker_output_list:
        lfid_idx = c.get_attr_names().index(c.get_property('foreign_key_ltable'))
        rfid_idx = c.get_attr_names().index(c.get_property('foreign_key_rtable'))

        for r in c.itertuples(index=False):
            id_set.append((r[lfid_idx], r[rfid_idx]))
    id_set = list(set(id_set))

    f_cols = fin_cols(l_col, r_col, ltable.get_key(), rtable.get_key())
    
    if len(id_set) > 0:
        id_df = pd.DataFrame(id_set)
        l_consol_table = l_df.ix[id_df[0]]
        r_consol_table = r_df.ix[id_df[1]]
        l_consol_table.reset_index(inplace=True, drop=True)
        r_consol_table.reset_index(inplace=True, drop=True)
        print l_consol_table.head(1)
        print r_consol_table.head(1)
        
        table = pd.concat([l_consol_table, r_consol_table], axis=1)
        table.sort([l_key, r_key], inplace=True)
        table.reset_index(inplace=True, drop=True)
        table = MTable(table[f_cols])
    else:
        table = MTable([], columns=f_cols)

    # project df and convert to MTable
    table.set_property('ltable', ltable)
    table.set_property('rtable', rtable)
    table.set_property('foreign_key_ltable', 'ltable.'+ltable.get_key())
    table.set_property('foreign_key_rtable', 'rtable.'+rtable.get_key())


    return table

time: 70.9 ms


In [5]:
A = mg.read_csv(mg.get_install_path() +'/datasets/DBLP_ACM/DBLP_cleaned.csv', key='id')

time: 17.9 ms


In [6]:
B = mg.read_csv(mg.get_install_path() + '/datasets/DBLP_ACM/ACM_cleaned.csv', key='id')

time: 13.8 ms


In [7]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(A, B, 'year', 'year', ['title', 'year', 'authors'], ['title', 'year', 'authors'])

time: 1.52 s


In [22]:
G1 = ab.block_tables(A, B, 'year', 'title')

time: 24.9 ms


In [23]:
G2 = ab.block_tables(A, B, 'year', 'authors', ['title', 'year', 'authors'], ['title', 'year', 'authors'])

time: 13.9 ms


In [28]:
G = cbn_via_union_v2([G2, G2])

time: 13.6 ms


In [29]:
G

,_id,ltable.id,rtable.id,ltable.authors,ltable.title,ltable.year,rtable.authors,rtable.title,rtable.year


time: 7.11 ms


In [9]:
op = mg.OverlapBlocker()
D = op.block_tables(A, B, 'authors', 'authors', l_output_attrs=['title', 'year', 'authors'], r_output_attrs=['title', 'year', 'authors'])

0%                          100%
[##############################] | ETA[sec]: 0.000 

time: 8.23 s



Total time elapsed: 5.744 sec


In [10]:
C1 = mg.create_mtable(C.ix[1:10000], ltable=A, rtable=B, foreign_key_ltable='ltable.id',
                     foreign_key_rtable='rtable.id')
D1 = mg.create_mtable(D.ix[1:10000], ltable=A, rtable=B, foreign_key_ltable='ltable.id',
                     foreign_key_rtable='rtable.id')
len(C1), len(D1)

(10000, 10000)

time: 101 ms


In [11]:
%load_ext line_profiler

time: 17.3 ms


In [13]:
# %lprun -f mg._combine_block_outputs_via_union  mg._combine_block_outputs_via_union([C1, D1])

time: 840 µs


In [12]:
E1 = mg.combine_block_outputs_via_union([C1, D1])

time: 18.6 s


In [15]:
E2 = cbn_via_union_v2([C1, D1])

                                     ltable.authors               ltable.id  \
0  Anindya Datta, Chris Jermaine, Edward Omiecinski  conf/vldb/JermaineDO99   

                                        ltable.title  ltable.year  
0  A Novel Index Supporting High Volume Data Ware...         1999  
                   rtable.authors  rtable.id  \
0  Tolga Bozkaya, Meral Ozsoyoglu     328959   

                                        rtable.title  rtable.year  
0  Indexing large metric spaces for similarity se...         1999  
time: 155 ms


In [16]:
E1.equals(E2)

True

time: 5.97 ms


In [17]:
E2.head()

,_id,ltable.id,rtable.id,ltable.authors,ltable.title,ltable.year,rtable.authors,rtable.title,rtable.year
0,0,conf/sigmod/AbadiC02,191844,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,2002,"Daniel Barbará, Tomasz Imieliński",Sleepers and workaholics: caching strategies i...,1994
1,1,conf/sigmod/AbadiC02,191882,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,2002,"Hwee Hwa Pang, Michael J. Carey, Miron Livny",Managing memory for real-time queries,1994
2,2,conf/sigmod/AbadiC02,191886,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,2002,"Jim Gray, Prakash Sundaresan, Susanne Englert,...",Quickly generating billion-record synthetic da...,1994
3,3,conf/sigmod/AbadiC02,191893,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,2002,"Hasanat M. Dewan, Salvatore J. Stolfo, Maurici...",Predictive dynamic load balancing of parallel ...,1994
4,4,conf/sigmod/AbadiC02,191911,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,2002,"Michael J. Carey, Michael J. Franklin, Markos ...",Fine-grained sharing in a page server OODBMS,1994


time: 18.4 ms


In [19]:
F = ab.block_candset(E2, 'title', 'title')

0%                          100%
[##############################] | ETA[sec]: 0.000 

time: 1.83 s



Total time elapsed: 1.818 sec


In [21]:
F.head()

,_id,ltable.id,rtable.id,ltable.authors,ltable.title,ltable.year,rtable.authors,rtable.title,rtable.year
106,106,conf/sigmod/AbadiC02,564770,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,2002,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,2002
990,990,conf/sigmod/AbiteboulBCMM03,872821,"Ioana Manolescu, Serge Abiteboul, Tova Milo, A...",Dynamic XML documents with distribution and re...,2003,"Serge Abiteboul, Angela Bonifati, Grégory Cobé...",Dynamic XML documents with distribution and re...,2003
4048,4048,conf/sigmod/Aggarwal02,564743,Charu C. Aggarwal,Hierarchical subspace sampling: a unified fram...,2002,Charu C. Aggarwal,Hierarchical subspace sampling: a unified fram...,2002
5647,5647,conf/sigmod/AgrawalCD02,564782,"Gautam Das, Sanjay Agrawal, Surajit Chaudhuri",DBXplorer: enabling keyword search over relati...,2002,"Sanjay Agrawal, Surajit Chaudhuri, Gautam Das",DBXplorer: enabling keyword search over relati...,2002
7797,7797,conf/sigmod/AltinelLKMPLWB02,564765,"Sailesh Krishnamurthy, C. Mohan, Larry Brown, ...",DBCache: database caching for web application ...,2002,"Mehmet Altinel, Qiong Luo, Sailesh Krishnamurt...",DBCache: database caching for web application ...,2002


time: 19.3 ms


In [18]:
E1.head()

,_id,ltable.id,rtable.id,ltable.authors,ltable.title,ltable.year,rtable.authors,rtable.title,rtable.year
0,0,conf/sigmod/AbadiC02,191844,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,2002,"Daniel Barbará, Tomasz Imieliński",Sleepers and workaholics: caching strategies i...,1994
1,1,conf/sigmod/AbadiC02,191882,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,2002,"Hwee Hwa Pang, Michael J. Carey, Miron Livny",Managing memory for real-time queries,1994
2,2,conf/sigmod/AbadiC02,191886,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,2002,"Jim Gray, Prakash Sundaresan, Susanne Englert,...",Quickly generating billion-record synthetic da...,1994
3,3,conf/sigmod/AbadiC02,191893,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,2002,"Hasanat M. Dewan, Salvatore J. Stolfo, Maurici...",Predictive dynamic load balancing of parallel ...,1994
4,4,conf/sigmod/AbadiC02,191911,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,2002,"Michael J. Carey, Michael J. Franklin, Markos ...",Fine-grained sharing in a page server OODBMS,1994


time: 32.3 ms
